In [ ]:
import pandas as pd
from gurobipy import Model, GRB, quicksum
from collections import defaultdict

def solve_vrp_subproblem(tasks_df, agvs_df, subproblem_name="Subproblem"):
    """
    Args:
        tasks_df (pd.DataFrame): 분할된 task 데이터
        agvs_df (pd.DataFrame): agv 데이터
        subproblem_name (str): 로그 출력을 위한 문제 이름

    Returns:
        dict: 해결된 경로, 비용, 지연 시간 등 결과 정보를 담은 딕셔너리
    """
    # === Parameters & sets ===
    depot = (0, 0)
    # 각 subproblem의 task 인덱스는 0부터 시작하므로, 노드 번호를 1부터 매깁니다.
    points = [(idx + 1, row['x'], row['y']) for idx, row in tasks_df.iterrows()]
    # 시작점(0)과 종료점(마지막 번호) 추가
    points = [(0, *depot)] + points + [(len(points) + 1, *depot)]

    distance = {
        i_idx: {
            j_idx: abs(i_x - j_x) + abs(i_y - j_y)
            for j_idx, j_x, j_y in points
        }
        for i_idx, i_x, i_y in points
    }

    NODE = len(tasks_df) + 2
    K = list(range(len(agvs_df)))
    N = list(range(1, len(points) - 1))       # 작업 노드 (depot 제외)
    N_s = list(range(0, len(points) - 1))     # 출발 가능 노드 (시작 depot + 작업)
    N_t = list(range(1, len(points)))         # 도착 가능 노드 (작업 + 종료 depot)
    N_s_t = list(range(0, len(points)))       # 모든 노드
    
    # 각 AGV가 수행할 수 있는 최대 경로(trip) 수
    T_max = [4 for _ in range(len(agvs_df))]

    v = agvs_df['speed_cells_per_sec'].tolist()
    Q = agvs_df['capacity'].tolist()
    D = agvs_df['max_distance'].tolist()
    
    # list index에 맞게 depot(0)에 대한 값 추가
    s = [0] + tasks_df['service_time'].tolist() + [0]
    u = [0] + tasks_df['demand'].tolist() + [0]
    deadlines = [0] + tasks_df['deadline'].tolist() + [0]
    
    alpha, beta = 1, 2

    # === Build Gurobi model ===
    model = Model(f"AGV_Routing_{subproblem_name}")
    x, y, o, t = {}, {}, {}, {}

    # -- Variables --
    for k in K:
        for r in range(T_max[k]):
            for i in N_s_t:
                for j in N_s_t:
                    if i != j:
                        x[k, i, j, r] = model.addVar(vtype=GRB.BINARY, name=f"x_{k}_{i}_{j}_{r}")
            for i in N_s_t:
                y[k, i, r] = model.addVar(vtype=GRB.BINARY, name=f"y_{k}_{i}_{r}")
                o[k, i, r] = model.addVar(vtype=GRB.INTEGER, name=f"o_{k}_{i}_{r}")
                t[k, i, r] = model.addVar(vtype=GRB.CONTINUOUS, name=f"t_{k}_{i}_{r}")

    late = {i: model.addVar(vtype=GRB.CONTINUOUS, name=f"late_{i}") for i in N}
    model.update()

    # === Constraints ===
    # 1. 각 AGV는 첫 번째 경로(trip 0)에서 depot을 한 번만 출발
    for k in K:
        for r in range(0,2):
            model.addConstr(quicksum(x[k, 0, j, r] for j in N) == 1, name=f"start_depot_{k}")

    # 2. 흐름 보존 (Flow conservation)
    for k in K:
        for r in range(T_max[k]):
            for j in N:
                model.addConstr(quicksum(x[k, i, j, r] for i in N_s if i != j) == quicksum(x[k, j, i, r] for i in N_t if i != j), name=f"flow_cons_{k}_{r}_{j}")

    # 3. y[k,j,r] 정의: j 노드를 k-agv가 r-경로에서 방문하면 1
    for k in K:
        for r in range(T_max[k]):
            for j in N_t: # 도착 가능한 모든 노드
                model.addConstr(quicksum(x[k, i, j, r] for i in N_s if i != j) == y[k, j, r], name=f"y_def_{k}_{r}_{j}")

    # 4. 시간 순서 제약 (Time sequencing) - Indicator Constraint 사용
    for k in K:
        for r in range(T_max[k]):
            for i in N_s:
                for j in N_t:
                    if i != j:
                        model.addConstr((x[k, i, j, r] == 1) >> (t[k, j, r] >= t[k, i, r] + distance[i][j] / v[k] + s[i]), name=f"time_seq_{k}_{r}_{i}_{j}")

    # 5. 용량 제약 (Capacity)
    for k in K:
        for r in range(T_max[k]):
            model.addConstr(quicksum(u[i] * y[k, i, r] for i in N) <= Q[k], name=f"capacity_{k}_{r}")

    # 6. 최대 이동 거리 제약 (Distance)
    for k in K:
        for r in range(T_max[k]):
            model.addConstr(quicksum(distance[i][j] * x[k, i, j, r] for i in N_s_t for j in N_s_t if i != j) <= D[k], name=f"max_dist_{k}_{r}")

    # 7. 모든 작업은 정확히 한 번씩 처리
    for j in N:
        model.addConstr(quicksum(y[k, j, r] for k in K for r in range(T_max[k])) == 1, name=f"serve_task_{j}")

    # 8. 시작 시간은 0
    for k in K:
        model.addConstr(t[k, 0, 0] == 0, name=f"start_time_{k}")

    # 9. 지연 시간(Lateness) 계산
    for i in N:
        for k in K:
            for r in range(T_max[k]):
                model.addConstr((y[k, i, r] == 1) >> (late[i] >= t[k, i, r] - deadlines[i]), name=f"lateness_calc_{i}_{k}_{r}")
    
    for i in N:
        model.addConstr(late[i] >= 0, name=f"lateness_nonneg_{i}")

    # 10. AGV의 연속된 경로(trip) 시간 제약
    for k in K:
        for r in range(1, T_max[k]):
            end_depot_node = N_s_t[-1] # 종료 depot의 노드 번호
            model.addConstr(t[k, end_depot_node, r - 1] <= t[k, 0, r], name=f"trip_continuity_{k}_{r}")

    # === Objective Function ===
    tardiness = quicksum(late[i] for i in N)
    travel_cost = quicksum(
        (distance[i][j] / v[k]) * x[k, i, j, r]
        for k in K for r in range(T_max[k])
        for i in N_s_t for j in N_s_t
        if i != j
    )
    model.addConstr(alpha * travel_cost + beta * tardiness >= 300)
    model.setObjective(alpha * travel_cost + beta * tardiness, GRB.MINIMIZE)

    # === Solve ===
    model.setParam(GRB.Param.TimeLimit, 60 * 20)
    model.optimize()

    # === Result Extraction ===
    results = {
        "solved": False,
        "routes": {},
        "travel_cost": float('inf'),
        "tardiness": float('inf'),
        "objective": float('inf')
    }
    
    if model.status == GRB.OPTIMAL or model.status == GRB.TIME_LIMIT or model.status == GRB.CUTOFF:
        if model.SolCount > 0:
            results["solved"] = True
            results["objective"] = model.ObjVal
            
            final_tardiness = sum(late[i].X for i in N)
            final_travel_cost = 0
            for k in K:
                for r in range(T_max[k]):
                    for i in N_s_t:
                        for j in N_s_t:
                            if i != j and x[k, i, j, r].X > 0.5:
                                final_travel_cost += (distance[i][j] / v[k])

            results["travel_cost"] = final_travel_cost
            results["tardiness"] = final_tardiness
            
            agv_routes = {k: {} for k in K}
            for k in K:
                for r in range(T_max[k]):
                    if r == 0 or any(x[k, 0, j, r].X > 0.5 for j in N):
                        current_route = []
                        current_node = 0
                        
                        for _ in range(len(N_s_t)):
                            current_route.append(current_node)
                            if current_node == N_s_t[-1]:
                                break
                            
                            found_next = False
                            for j_node in N_t:
                                if current_node != j_node and x[k, current_node, j_node, r].X > 0.5:
                                    current_node = j_node
                                    found_next = True
                                    break
                            if not found_next:
                                break
                        
                        if len(current_route) > 1 and not (current_route[0]==0 and current_route[-1]==N_s_t[-1] and len(current_route) == 2):
                             agv_routes[k][r] = current_route

            results["routes"] = agv_routes
        else:
            print(f"[{subproblem_name}] Solution not found within time limit or cutoff.")
    else:
        print(f"[{subproblem_name}] Optimization finished with status: {model.status}")

    return results

if __name__ == "__main__":
    # --- 1. 데이터 로드 및 분할 ---
    try:
        all_tasks = pd.read_csv("./open_v2/task.csv")
        all_agvs = pd.read_csv("./open_v2/agv.csv")
    except FileNotFoundError:
        exit()

    # Task를 4분할
    num_tasks = len(all_tasks)
    s1_t = num_tasks // 4
    s2_t = 2 * (num_tasks // 4)
    s3_t = 3 * (num_tasks // 4)
    
    tasks_part1 = all_tasks.iloc[:s1_t].copy()
    tasks_part2 = all_tasks.iloc[s1_t:s2_t].copy()
    tasks_part3 = all_tasks.iloc[s2_t:s3_t].copy()
    tasks_part4 = all_tasks.iloc[s3_t:].copy()
    
    tasks_part2.reset_index(drop=True, inplace=True)
    tasks_part3.reset_index(drop=True, inplace=True)
    tasks_part4.reset_index(drop=True, inplace=True)
    
    # AGV를 4분할
    num_agvs = len(all_agvs)
    s1_a = num_agvs // 4
    s2_a = 2 * (num_agvs // 4)
    s3_a = 3 * (num_agvs // 4)

    agvs_part1 = all_agvs.iloc[:4].copy()
    agvs_part2 = all_agvs.iloc[4:8].copy()
    agvs_part3 = all_agvs.iloc[8:12].copy()
    agvs_part4 = all_agvs.iloc[12:].copy()

    agvs_part2.reset_index(drop=True, inplace=True)
    agvs_part3.reset_index(drop=True, inplace=True)
    agvs_part4.reset_index(drop=True, inplace=True)

    print("Data loading and splitting into 4 parts complete.")
    print(f"Part 1: {len(tasks_part1)} tasks, {len(agvs_part1)} AGVs")
    print(f"Part 2: {len(tasks_part2)} tasks, {len(agvs_part2)} AGVs")
    print(f"Part 3: {len(tasks_part3)} tasks, {len(agvs_part3)} AGVs")
    print(f"Part 4: {len(tasks_part4)} tasks, {len(agvs_part4)} AGVs")

    # --- 2. 각 하위 문제 해결 ---
    solution1 = solve_vrp_subproblem(tasks_part1, agvs_part1, "Part 1")
    solution2 = solve_vrp_subproblem(tasks_part2, agvs_part2, "Part 2")
    solution3 = solve_vrp_subproblem(tasks_part3, agvs_part3, "Part 3")
    solution4 = solve_vrp_subproblem(tasks_part4, agvs_part4, "Part 4")

    # --- 3. 결과 통합 및 출력 ---
    print("\n\n" + "="*50)
    print("           FINAL COMBINED RESULTS")
    print("="*50)

    all_agv_full_paths = defaultdict(list)
    solutions = [solution1, solution2, solution3, solution4]
    task_parts = [tasks_part1, tasks_part2, tasks_part3, tasks_part4]
    agv_parts = [agvs_part1, agvs_part2, agvs_part3, agvs_part4]
    
    task_offset = 0
    agv_offset = 0

    for i, sol in enumerate(solutions):
        part_num = i + 1
        print(f"\n--- Solution for Part {part_num} (with original task IDs) ---")
        if sol["solved"]:
            print(f"Objective: {sol['objective']:.2f}, Travel Cost: {sol['travel_cost']:.2f}, Tardiness: {sol['tardiness']:.2f}")
            tasks_df = task_parts[i]
            
            for k_local, trips in sol["routes"].items():
                if trips:
                    global_k = k_local + agv_offset
                    print(f"  AGV {global_k}:")
                    for r in sorted(trips.keys()):
                        route = trips[r]
                        mapped_task_ids = []
                        
                        # Task ID 매핑
                        for node in route:
                            if node != 0 and node != len(tasks_df) + 1:
                                temp_index = node - 1
                                original_task_index = temp_index + task_offset
                                mapped_task_ids.append(original_task_index)
                        print(f"    Trip {r}: Depot -> {' -> '.join(map(str, mapped_task_ids))} -> Depot")

                        # CSV 저장을 위한 경로 포맷팅
                        formatted_trip = []
                        for node in route:
                            if node == 0 or node == len(tasks_df) + 1:
                                formatted_trip.append("DEPOT")
                            else:
                                temp_index = node - 1
                                original_task_index = temp_index + task_offset
                                formatted_trip.append(f"T{original_task_index:04d}")
                        all_agv_full_paths[global_k].extend(formatted_trip)
        else:
            print(f"No solution found for Part {part_num}.")
        
        # 다음 파트를 위한 오프셋 누적
        task_offset += len(task_parts[i])
        agv_offset += len(agv_parts[i])


    # --- 4. 최종 경로 CSV 파일로 저장 ---
    if any(sol["solved"] for sol in solutions):
        final_csv_data = []
        for k in sorted(all_agv_full_paths.keys()):
            full_path = all_agv_full_paths[k]
            clean_path = [full_path[0]] if full_path else []
            for node in full_path[1:]:
                if node != "DEPOT" or clean_path[-1] != "DEPOT":
                    clean_path.append(node)
            
            final_csv_data.append({
                "agv_id": f"A{k+1:03d}",
                "route": ",".join(clean_path)
            })

        path_df = pd.DataFrame(final_csv_data)
        output_path = "0930_agv_routes_formatted_4_parts.csv"
        path_df.to_csv(output_path, index=False)
        print(f"AGV 경로가 CSV 파일로 저장되었습니다: {output_path}")
    else:
        print("최적 해를 찾지 못했기 때문에 경로를 저장하지 않습니다.")

    # -- 전체 결과 요약 --
    total_objective = sum(s['objective'] for s in solutions if s['solved'])
    total_cost = sum(s['travel_cost'] for s in solutions if s['solved'])
    total_tardiness = sum(s['tardiness'] for s in solutions if s['solved'])

    print("\n--- Overall Summary ---")
    print(f"Total Estimated Objective: {total_objective:.2f}")
    print(f"Total Travel Cost: {total_cost:.2f}")
    print(f"Total Tardiness: {total_tardiness:.2f}")
    print("="*50)

Data loading and splitting into 4 parts complete.
Part 1: 25 tasks, 4 AGVs
Part 2: 25 tasks, 4 AGVs
Part 3: 25 tasks, 4 AGVs
Part 4: 25 tasks, 3 AGVs

--- [Part 1] 모델링 및 최적화 시작 ---
Tasks: 25, AGVs: 4
Set parameter Username
Set parameter LicenseID to value 2636023
Academic license - for non-commercial use only - expires 2026-03-13
Set parameter TimeLimit to value 1200
Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (win64 - Windows 11.0 (26100.2))

CPU model: Intel(R) Core(TM) i7-14700, instruction set [SSE2|AVX|AVX2]
Thread count: 20 physical cores, 28 logical processors, using up to 28 threads

Non-default parameters:
TimeLimit  1200

Optimize a model with 923 rows, 12553 columns and 54310 nonzeros
Model fingerprint: 0x16eda8f8
Model has 10816 simple general constraints
  10816 INDICATOR
Variable types: 457 continuous, 12096 integer (11664 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+02]
  Objective range  [1e+00, 1e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range 

In [ ]:
# 문자열 데이터를 DataFrame으로 읽어옵니다.
df = path_df

# 'route' 열의 각 경로를 처리하는 함수를 정의합니다.
def increment_task_numbers(route_string):
    """경로 문자열을 받아서 작업 번호에 1을 더한 새로운 문자열을 반환합니다."""
    parts = route_string.split(',')
    new_parts = []
    for part in parts:
        # 'T'로 시작하는 작업 번호인지 확인합니다.
        if part.startswith('T'):
            try:
                # 숫자 부분을 추출하고 정수로 변환합니다.
                task_number = int(part[1:])
                # 1을 더한 후, 다시 'Txxxx' 형식의 문자열로 만듭니다.
                new_part = f"T{task_number + 1:04d}"
                new_parts.append(new_part)
            except (ValueError, IndexError):
                # 숫자 변환에 실패하면 원본을 그대로 유지합니다.
                new_parts.append(part)
        else:
            # 'DEPOT' 등 다른 문자열은 그대로 유지합니다.
            new_parts.append(part)
    
    # 처리된 부분들을 다시 쉼표로 연결하여 반환합니다.
    return ",".join(new_parts)

# apply 함수를 사용하여 'route' 열에 함수를 일괄 적용합니다.
df['route'] = df['route'].apply(increment_task_numbers)

# 결과를 새로운 CSV 파일로 저장합니다.
output_filename = 'updated_agv_routes_last_10.csv'
df.to_csv(output_filename, index=False)


✅ 경로의 작업 번호를 업데이트하여 'updated_agv_routes_last_10.csv' 파일로 저장했습니다.
